In [2]:
import math

In [3]:
import ee
ee.Initialize()

In [4]:
chirpsDaily = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
chirpsPentads = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD")

chirpsInUse = chirpsPentads

In [5]:
CLOUDBUCKET = "ee-oxford-upload"

In [6]:
dependentVar = "precipitation"
nCycles = 3
cycles = ee.List.sequence(1, nCycles)
cycles.getInfo()

[1.0, 2.0, 3.0]

In [65]:
cosNames = ["cos_" + str(i) for i in range(1, nCycles+1)]
sinNames = ["sin_" + str(i) for i in range(1, nCycles+1)]
independents = ['offset', 'slope']
independents.extend(cosNames)
independents.extend(sinNames)

In [66]:
independents

['offset', 'slope', 'cos_1', 'cos_2', 'cos_3', 'sin_1', 'sin_2', 'sin_3']

In [36]:
def listBandNames(img):
    u =  ee.Image(fittedModel.first()).bandNames().getInfo()
    return[str(uname) for uname in u]    

In [73]:
# adds constant bands to an image, one for the time of year of this image 
# in terms of radians and another with constant value 1
def addDependents(image):
    years = image.date().difference('2000-01-01', 'year');
    timeRadians = ee.Image(years.multiply(2 * math.pi)).rename(['slope'])
    constant = ee.Image(1).rename(['offset'])
    return image.addBands(constant).addBands(timeRadians.float())

# adds harmonic bands to the image, i.e. the cos(t) and sin(t) and cos(2t) and sin(2t) etc
# of the image's time - i.e. these bands are again each constant values across an image
def addHarmonics(frequencies):
    def helper(image):
        # make a constant value image with one band for each member of the frequencies list
        # note that frequencies will be available to this returned function via a closure
        freqsImg = ee.Image.constant(frequencies)
        time = ee.Image(image).select('slope')
        cosines = time.multiply(freqsImg).cos().rename(cosNames)
        sines = time.multiply(freqsImg).sin().rename(sinNames)
        return image.addBands(cosines).addBands(sines)
    return helper

addHarmonicsFn = addHarmonics(cycles)

In [75]:
chirpsWithHarmonics = chirpsInUse.map(addDependents).map(addHarmonicsFn)

In [76]:
# not independents.append(dependentVar) - do this instead to make a copy so we 
# still have independents unmodified
regressionBandList = independents + [dependentVar]

In [77]:
# select the bands: this is to ensure they're in the correct order as the reducer will expect the 
# independents to be in order and followed by the dependents, in the numbers specified
orderedBands = chirpsWithHarmonics.select(regressionBandList)
harmonicTrends = orderedBands.reduce(ee.Reducer.linearRegression(len(independents), 1))

The harmonicTrends output from the reducer is a two-banded array image, that is, every pixel in each band is an array with dimensions (numX, numY) i.e. number of independents * number of dependents, i.e. in this case 8x1 (one constant, one time band, 3 each cos and sin bands)

The first band represents the coefficients of the linear regression and the second represents the rms of the residuals.

Array images are quite awkward so first off make it back into a "normal" image where each pixel in each band has one value

In [78]:
# Turn the array image into an image with one band for each coefficient, i.e. 8 bands
harmonicTrendCoefficients = harmonicTrends.select('coefficients').arrayProject([0]).arrayFlatten([independents])

Now apply the model by multiplying the independent variables by their coefficients and adding them; add the fitted result prediction as a new band to the same image collection

In [79]:
# Add the result to the main image collection so now we have everything in that one collection
fittedModel = chirpsWithHarmonics.map(
    lambda img: img.addBands(
        img.select(independents).multiply(harmonicTrendCoefficients).reduce('sum').rename(['fitted'])))
print(listBandNames(fittedModel.first()))

['precipitation', 'offset', 'slope', 'cos_1', 'cos_2', 'cos_3', 'sin_1', 'sin_2', 'sin_3', 'fitted']


In [18]:
nationalBoundaries = ee.FeatureCollection("USDOS/LSIB/2013")
ISO3 = "BWA";
geoms = nationalBoundaries.filterMetadata("iso_alpha3", "equals", ISO3);
bufferedboxes = geoms.map(lambda f : f.bounds().buffer(10000).bounds())
roi = bufferedboxes.union(ee.ErrorMargin(10));

In [38]:
s = fittedModel.select(['fitted', 'precipitation'])#, roi, ee.Reducer.mean(), 10000)
print(listBandNames(s.first()))

['precipitation', 'constant', 't', 'cos_1', 'cos_2', 'cos_3', 'sin_1', 'sin_2', 'sin_3', 'fitted']


In [31]:
test = ee.Image(s.first())

In [58]:
test.reduceRegion(ee.Reducer.mean(), roi, 10000).getInfo()

{u'fitted': 14.240612458246016, u'precipitation': 4.43393421765722}

In [61]:
independents[1]='offset'

In [62]:
independents

['constant', 'offset', 'cos_1', 'cos_2', 'cos_3', 'sin_1', 'sin_2', 'sin_3']

In [42]:
for cName, sName in zip(cosNames, sinNames):
    phaseNum = cName[-1:]
    cosBand = harmonicTrendCoefficients.select(cName)
    sinBand = harmonicTrendCoefficients.select(sName)
    # magnitude depends on the scale of the source data really, not appropriate to attempt to scale it 
    # withoug assessing the max and min values across a ROI
    cycleN_Magnitude = cosBand.hypot(sinBand).rename(['mag_' + phaseNum])
    # phase is by definition between -pi and +pi so we can scale that to 0-1 for a time-of-year of peak
    cycleN_Phase = sinBand.atan2(cosBand).unitScale(-math.pi, math.pi).rename(['phase_' + phaseNum])
    
    

('cos_1', 'sin_1', '1')
('cos_2', 'sin_2', '2')
('cos_3', 'sin_3', '3')


In [43]:
cosBand = harmonicTrendCoefficients.select("cos_1")
sinBand = harmonicTrendCoefficients.select("sin_1")
cycleN_Magnitude = cosBand.hypot(sinBand).multiply(5).rename(['mag_1'])

In [55]:
cycleN_Magnitude.reduceRegion(ee.Reducer.minMax(), roi, 10000, 'EPSG:4326').getInfo()

{u'mag_1_max': 78.72957107399895, u'mag_1_min': 10.288261478141603}

In [56]:
cosBand.reduceRegion(ee.Reducer.minMax(), roi, 10000, 'EPSG:4326').getInfo()

{u'cos_1_max': 15.222657203674316, u'cos_1_min': 1.5761271715164185}

In [57]:
sinBand.reduceRegion(ee.Reducer.minMax(), roi, 10000, 'EPSG:4326').getInfo()

{u'sin_1_max': 5.055975437164307, u'sin_1_min': -1.4135379791259766}

In [16]:
l.getInfo()

[1, 2, 3]

In [40]:
import math

def unpack(thelist):
    unpacked = []
    for i in thelist:
        unpacked.append(i[0])
        unpacked.append(i[1])
    return unpacked

# bbox if specified must be a list of 4 coords [xmin, ymin, xmax, ymax]
def buildExtractionParams(iso3=None, bbox=None, 
                          metresResolution=None, pixPerDegreeResolution=None):
    # this works out to be a boolean xor test
    if (iso3 is not None) == (bbox is not None):
        raise ValueError("either ISO3 OR a bounding box must be specified but not both")
    if (metresResolution is not None) == (pixPerDegreeResolution is not None):
        raise ValueError("either a pixel resolution in metres or in exact pixels per degree must be specified but not both")

    output = {
        #image, description, bucket, fileNamePrefix, maxPixels, region, dimensions OR scale, crs
        'crs': 'EPSG:4326',
        'maxPixels': 400000000,
        'bucket': CLOUDBUCKET
    }
    
    if iso3 is not None:
        # Get a geometry for the area of interest 
        geoms = nationalBoundaries.filterMetadata("iso_alpha3", "equals", ISO3_TO_EXPORT);
        bufferedboxes = geoms.map(lambda f: (f.bounds().buffer(10000).bounds()))
  
        roi = bufferedboxes.union(ee.ErrorMargin(10)).first()
        #https://groups.google.com/d/msg/google-earth-engine-developers/TViMuO3ObeM/cpNNg-eMDAAJ
        roiCoordsBizarrelyRequired = combinedboxes.getInfo()['geometry']['coordinates']
    else:
        roi = ee.Geometry.Rectangle(bbox)
        # the export call needs a json-serializable object, not an actual ee geometry object, this is different from JS i think
        roiCoordsBizarrelyRequired = roi.getInfo()['coordinates']
    output['region_geom'] = roi
    output['region'] = roiCoordsBizarrelyRequired
    if metresResolution is not None:
        output['scale'] = metresResolution
    else:
        if bbox is None:
            # we need to get the xmin, xmax, ymin, ymax coords from the country geometry object, this seems to be 
            # bizarrely difficult unless i am missing something
            # create a flat list of [x, y, x1, y1, x2, y2, ...]
            unpackedCoords = unpack(roiCoordsBizarrelyRequired[0])
            xCoords = unpackedCoords[::2]
            yCoords = unpackedCoords[1:][::2]
            xmin = min(xCoords)
            xmax = max(xCoords)
            ymin = min(yCoords)
            ymax = max(yCoords)
        else:
            xmin, ymin, xmax, ymax = bbox
        xDimFrac = (xmax-xmin) * pixPerDegreeResolution
        yDimFrac = math.ceil((ymax-ymin) * pixPerDegreeResolution)
        if (int(xDimFrac) != xDimFrac) or (int(yDimFrac)!= yDimFrac):
            print("Warning - non integer number of pixels fits, will round up")
        xDim = int(math.ceil(xDimFrac))
        yDim = int(math.ceil(yDimFrac))
        dimensionsString = str(xDim)+"x"+str(yDim)
        output['dimensions'] = dimensionsString
    return output

# call the export routine; same function whichever way we are defining the output size
def exportFunction(prebakedParams, prefix, description, image):
    # copy the input so it isn't modified (pass-by-reference), just in case of confusion - this
    # won't make a copy of the region as that's a list but we're not going to change that so 
    # it doesn't really matter
    exportParams=prebakedParams.copy()
    exportParams['image']=image
    exportParams['fileNamePrefix']=prefix
    exportParams['description']=description
    task = ee.batch.Export.image.toCloudStorage(**exportParams)
    return task